In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
url = 'https://therooftalks.com/page/17/'
r = requests.get(url)

In [4]:
soup = BeautifulSoup(r.content, "lxml")

In [8]:
a = soup.find_all('div', {'class': 'article--grid__header'})
a = [a_.find('a').get('href') for a_ in a]
a

['https://therooftalks.com/2016/11/30/best-infinity-pools-in-kuala-lumpur/',
 'https://therooftalks.com/2016/11/25/home-improvement-3-decluttering-hacks-for-weekend/',
 'https://therooftalks.com/2016/11/23/consider-a-home-in-johor-bahru/',
 'https://therooftalks.com/2016/08/04/how-find-best-property-agent/',
 'https://therooftalks.com/2016/08/03/how-pokemon-go-can-help-property-agents/',
 'https://therooftalks.com/2016/07/28/6-tips-to-negotiate-with-an-agent/',
 'https://therooftalks.com/2016/07/01/new-vs-old-decide-which-to-buy/',
 'https://therooftalks.com/2016/06/21/guide-buying-first-home-kl/']

In [10]:
def generator():
    while True:
        yield
        
index = 1
all_a = []

for _ in tqdm(generator()):
    url = f'https://therooftalks.com/page/{index}/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    a = soup.find_all('div', {'class': 'article--grid__header'})
    a = [a_.find('a').get('href') for a_ in a]
    
    if not len(a):
        break
        
    all_a.extend(a)
    
    index += 1

17it [00:03,  4.48it/s]


In [11]:
urls = sorted(list(set(all_a)))
len(urls)

168

In [12]:
urls[:10]

['https://therooftalks.com/2016/06/21/guide-buying-first-home-kl/',
 'https://therooftalks.com/2016/07/01/new-vs-old-decide-which-to-buy/',
 'https://therooftalks.com/2016/07/28/6-tips-to-negotiate-with-an-agent/',
 'https://therooftalks.com/2016/08/03/how-pokemon-go-can-help-property-agents/',
 'https://therooftalks.com/2016/08/04/how-find-best-property-agent/',
 'https://therooftalks.com/2016/11/23/consider-a-home-in-johor-bahru/',
 'https://therooftalks.com/2016/11/25/home-improvement-3-decluttering-hacks-for-weekend/',
 'https://therooftalks.com/2016/11/30/best-infinity-pools-in-kuala-lumpur/',
 'https://therooftalks.com/2016/12/02/world-design-capital-2016-taipei/',
 'https://therooftalks.com/2016/12/06/good-relationship-with-your-agent/']

In [15]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

In [16]:
soup.find('h1').text.strip()

'Guide to buying your first home in KL'

In [18]:
soup.find('article', {'class': 'post-article'}).get_text(separator="\n")

'\n\nIt’s finally \nthat\n time in your life where you start looking for your own home. Even before you begin, you’re already thinking who to invite for your housewarming party. You’re looking forward to the trips to IKEA. You’re excited to meet your soon-to-be neighbors. You can’t wait to finally have your own home.\n\n\n\xa0\n\n\nBut\n where in the world do you begin\n?\n\n\n\n\nGoogle directs you to a list of websites which match your query “homes in Kuala Lumpur”. You spend an entire afternoon browsing through the pages of listings on these websites, feeling drained at the end of the whole process when you are still unable to find a property which fits your requirements. Your brain is cluttered with questions: \nHow do mortgages work? Are there government schemes to help me buy my first home? Are there additional fees I need to pay? For my salary, what kind of prices should I be looking at?\n\n\nIn this blog post, we help you answer some of these questions and walk you through the 

In [19]:
data = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        data.append({
            'url': url,
            'title': soup.find('h1').text.strip(),
            'body': soup.find('article', {'class': 'post-article'}).get_text(separator="\n"),
        })
    except:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [00:26<00:00,  6.24it/s]


In [20]:
data[0]

{'url': 'https://therooftalks.com/2016/06/21/guide-buying-first-home-kl/',
 'title': 'Guide to buying your first home in KL',
 'body': '\n\nIt’s finally \nthat\n time in your life where you start looking for your own home. Even before you begin, you’re already thinking who to invite for your housewarming party. You’re looking forward to the trips to IKEA. You’re excited to meet your soon-to-be neighbors. You can’t wait to finally have your own home.\n\n\n\xa0\n\n\nBut\n where in the world do you begin\n?\n\n\n\n\nGoogle directs you to a list of websites which match your query “homes in Kuala Lumpur”. You spend an entire afternoon browsing through the pages of listings on these websites, feeling drained at the end of the whole process when you are still unable to find a property which fits your requirements. Your brain is cluttered with questions: \nHow do mortgages work? Are there government schemes to help me buy my first home? Are there additional fees I need to pay? For my salary, w

In [21]:
with open('therooftalks.com.jsonl', 'w') as fopen:
    for d in data:
        fopen.write(f'{json.dumps(d)}\n')